# Analyzing Twitter data

## Collecting data through the Twitter API

APIs, or Application Programming Interfaces, are methods of accessing data from business of government organization.

### Twitter APIs

Twitter has multiple APIs which can be used for different purposes. These change from time to time and Twitter sometimes adds or removes APIs.

Twitter APIs include:

* Search API (allows access to tweets for the past week)

* Ads API (focuses on Twitter adds)

* Streaming API (allows to collect a sample of tweets in real-time based on keywords, user IDs, and locations)

### Streaming API

Streaming API allows you to collect a sample of tweets in real-time based on keywords, user IDs, and locations.

The connection to streaming API remains open until you close it.

The streaming API has 2 endpoints:

**filter endpoint:**

* keywords (few hundred)
   
* usernames (a few thousands)
    
* location ranges (25)


**sample endpoint:**

* random 1% sample of all of Twitter

### Collecting data from Streaming API using `tweepy`

To collect data from streaming API you can use a package called `tweepy`

`tweepy` abstract away much of the work we need to set up a stable Twitter Streaming API connection.